In [319]:
#Tensors and operations
import tensorflow as tf

tf.constant([[1.,2.,3.], [4.,5.,6.]]) #matrix
tf.constant(42) #scalar

t = tf.constant([[1.,2.,3.], [4.,5.,6.]])
t.shape
t.dtype
t[:, 1:]
t + 10
tf.square(t)
t @ tf.transpose(t) #@ is for matrix multiplication



<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [320]:
#Keras backend
from tensorflow import keras

K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

In [321]:
#Tensors and Numpy
import numpy as np

a = np.array([2.,4.,5.,])
tf.constant(a)
t.numpy()
tf.square(a)
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [322]:
#Type conversions and variables
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.) + tf.cast(t2, tf.float32)

v = tf.Variable([[1.,2.,3.,], [4.,5.,6.]])
v.assign(2*v)
v[0,1].assign(42)
v[:,2].assign([0.,1.])
v.scatter_nd_update(indices=[[0,0],[1,2]], updates=[100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

In [323]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [324]:
#Customizing Models and Training Algorithims

#Custom Loss Functions

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', kernel_initializer='lecun_normal', input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=huber_fn, optimizer='nadam', metrics=['mae'])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))


Epoch 1/2


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.1011 - mae: 1.5234 - val_loss: 0.2934 - val_mae: 0.5872
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2358 - mae: 0.5359 - val_loss: 0.2458 - val_mae: 0.5337


In [325]:
model.save('model_with_custom_loss.h5')

model = keras.models.load_model('model_with_custom_loss.h5',
                                custom_objects={'huber_fn': huber_fn})

model.compile(loss=huber_fn, optimizer='nadam', metrics=['mae'])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2198 - mae: 0.5145 - val_loss: 0.2065 - val_mae: 0.4911
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2119 - mae: 0.5044 - val_loss: 0.1902 - val_mae: 0.4734


In [326]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < 1
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer='nadam', metrics=['mae'])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

model.save("my_model_with_a_custom_loss_threshold_2.h5")

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2066 - mae: 0.4977 - val_loss: 0.1902 - val_mae: 0.4722
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2033 - mae: 0.4931 - val_loss: 0.1854 - val_mae: 0.4661


In [327]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

model.compile(loss=create_huber(2.0), optimizer='nadam', metrics=['mae'])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2002 - mae: 0.4890 - val_loss: 0.1887 - val_mae: 0.4684
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1977 - mae: 0.4851 - val_loss: 0.1827 - val_mae: 0.4615


In [328]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 /2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.7223 - mae: 1.5410 - val_loss: 0.3285 - val_mae: 0.5692
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2700 - mae: 0.5408 - val_loss: 0.2475 - val_mae: 0.4982


In [329]:
model.save("my_model_with_a_custom_loss_class.h5")

In [330]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

In [331]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Set learning rate or any other parameter as needed
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [332]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0020 - loss: -17.5692 - val_accuracy: 0.0044 - val_loss: -16.6890
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0020 - loss: -17.6813 - val_accuracy: 0.0044 - val_loss: -16.7223


In [333]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [334]:
#Custom Activation Functions, Initializers, Regularizers, and Constraints
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [335]:
from tensorflow.keras import regularizers

class MyL1Regularizer(regularizers.Regularizer):
    def __init__(self, l1=0.01):
        self.l1 = l1
    def __call__(self, weights):
        return self.l1 * tf.reduce_sum(tf.abs(weights))
    def get_config(self):
        return {"l1": self.l1}

my_l1_regularizer = MyL1Regularizer(l1=0.01)

In [336]:
from tensorflow.keras import constraints

class MyPositiveWeightsConstraint(constraints.Constraint):
    def __call__(self, weights):
        return tf.where(weights < 0, tf.zeros_like(weights), weights)

my_positive_weights = MyPositiveWeightsConstraint()

In [337]:
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer = my_glorot_initializer,
                           kernel_regularizer = my_l1_regularizer,
                           kernel_constraint = my_positive_weights)

In [338]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [339]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', kernel_initializer='lecun_normal',
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer = my_glorot_initializer,
                           kernel_regularizer = my_l1_regularizer,
                           kernel_constraint = my_positive_weights)
])

In [340]:
model.compile(loss='mse', optimizer='nadam', metrics=['mae'])

In [341]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 2.5945 - mae: 1.1947 - val_loss: inf - val_mae: inf
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7183 - mae: 0.5367 - val_loss: inf - val_mae: inf


In [342]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [343]:
#Custom metrics
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - huber_fn: 1.1750 - loss: 4.4650
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - huber_fn: 0.2373 - loss: 0.5620


In [344]:

sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled, y_train, epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - huber_fn: 0.2171 - loss: 0.2493
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - huber_fn: 0.2137 - loss: 0.2441


In [345]:
history.history["loss"][0], history.history["huber_fn"][0] * sample_weight.mean()

(0.24019309878349304, 0.10391573643366168)

In [346]:
#Streaming metrics
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [347]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])
precision.result()
precision.variables
precision.reset_state()

In [348]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight(name="total", initializer="zeros", shape=())
        self.count = self.add_weight(name="count", initializer="zeros", shape=())
    def update_state(self, y_true, y_pred):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [349]:
m = HuberMetric(2.)
m(tf.constant([[2.]]), tf.constant([[10.]]))

<tf.Tensor: shape=(), dtype=float32, numpy=7.5>

In [350]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        batch_input_shape = tf.TensorShape(batch_input_shape)
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [351]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [352]:
model = keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=input_shape),
    MyDense(1)
])

model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

model.fit(X_train_scaled, y_train, epochs=2)
model.evaluate(X_test_scaled, y_test)

Epoch 1/2


/var/folders/pn/dyw6gjhx64x2qf0zzv2qx82h0000gn/T/ipykernel_88243/1038721765.py:3: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - huber_fn: 0.7990 - loss: 2.5178
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - huber_fn: 0.2663 - loss: 0.6492
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - huber_fn: 0.2125 - loss: 0.4936


[0.4896085858345032, 0.21101237833499908]

In [353]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        print("X1.shape: ", X1.shape ," X2.shape: ", X2.shape) # Debugging of custom layer
        return X1 + X2, X1 * X2

    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

In [354]:
inputs1 = keras.layers.Input(shape=[2])
inputs2 = keras.layers.Input(shape=[2])
outputs1, outputs2 = MyMultiLayer()((inputs1, inputs2))

In [355]:
def split_data(data):
    columns_count = data.shape[-1]
    half = columns_count // 2
    return data[:, :half], data[:, half:]

X_train_scaled_A, X_train_scaled_B = split_data(X_train_scaled)
X_valid_scaled_A, X_valid_scaled_B = split_data(X_valid_scaled)
X_test_scaled_A, X_test_scaled_B = split_data(X_test_scaled)

# Printing the splitted data shapes
X_train_scaled_A.shape, X_train_scaled_B.shape

((11610, 4), (11610, 4))

In [356]:
outputs1, outputs2 = MyMultiLayer()((X_train_scaled_A, X_train_scaled_B))

X1.shape:  (11610, 4)  X2.shape:  (11610, 4)


In [357]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

input_A = keras.layers.Input(shape=(X_train_scaled_A.shape[-1],))
input_B = keras.layers.Input(shape=(X_train_scaled_B.shape[-1],))
hidden_A, hidden_B = MyMultiLayer()((input_A, input_B))
hidden_A = keras.layers.Dense(30, activation='selu')(hidden_A)
hidden_B = keras.layers.Dense(30, activation='selu')(hidden_B)
concat = keras.layers.Concatenate()((hidden_A, hidden_B))
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

model.compile(loss='mse', optimizer='nadam')

model.fit((X_train_scaled_A, X_train_scaled_B), y_train, epochs=2,
          validation_data=((X_valid_scaled_A, X_valid_scaled_B), y_valid))

Epoch 1/2
X1.shape:  (None, 4)  X2.shape:  (None, 4)
X1.shape:  (None, 4)  X2.shape:  (None, 4)
341/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.3099X1.shape:  (None, 4)  X2.shape:  (None, 4)
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3.2219 - val_loss: 1.0741
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0080 - val_loss: 0.9879


In [358]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [359]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    AddGaussianNoise(stddev=1.0),
    keras.layers.Dense(30, activation="selu"),
    keras.layers.Dense(1)
])

In [360]:

model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3.9485 - val_loss: 0.8425
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0482 - val_loss: 0.7796
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - loss: 0.7717


0.7702970504760742

In [361]:
X_new_scaled = X_test_scaled

In [362]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [363]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [364]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [365]:
model = ResidualRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 136.2583
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3.3821
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4858
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1137
Epoch 5/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7303
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: 0.6306
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [366]:
class ReconstructingRegressor(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.dense = tf.keras.layers.Dense(units)
        self.out = tf.keras.layers.Dense(1)
    
    def call(self, inputs, training=False):
        Z = self.dense(inputs)
        recon_loss = tf.reduce_mean(tf.square(inputs - Z))
        
        if training:
            # If needed, you can use recon_loss here without additional functions
            result = tf.reduce_mean(recon_loss)  # Just as an example, you can process it as required
            # Optionally log recon_loss, but without add_metric since it's deprecated
        return self.out(Z)

# Compile the model and define metrics in compile
model = ReconstructingRegressor(1)
model.compile(loss="mse", optimizer="nadam", metrics=[tf.keras.metrics.MeanSquaredError()])



In [367]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [368]:
model = ReconstructingRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2)
y_pred = model.predict(X_test_scaled)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - loss: 6.4939
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - loss: 3.9547
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step


In [369]:
#Computing Gradients using Autodiff

def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

w1, w2 = 5, 3
lr = 0.00001
(f(w1 + lr, w2) - f(w1, w2)) / lr


36.00002999917251

In [370]:
(f(w1, w2 + lr) - f(w1, w2)) / lr

10.000000000331966

In [371]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1,w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [372]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z,w1)
dz_dw2 = tape.gradient(z,w2)

In [373]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients = tape.gradient(z, [c1,c2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [374]:
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
    z = f(w1,w2)

gradients = tape.gradient(z, [w1,w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

In [375]:
x= tf.Variable([100.])
with tf.GradientTape() as tape:
    z = my_softplus(x)

tape.gradient(z, [x])

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([nan], dtype=float32)>]

In [376]:
@tf.custom_gradient
def my_better_softplus(z):
    exp = np.exp(z)
    def my_softplus_gradients(grad):
        return grad / (1 + 1 / exp)
    return tf.math.log(exp+1), my_softplus_gradients

x = tf.Variable([1000.])
with tf.GradientTape() as tape:
    z = my_better_softplus(x)

z, tape.gradient(z, [x])

/var/folders/pn/dyw6gjhx64x2qf0zzv2qx82h0000gn/T/ipykernel_88243/2892339465.py:3: RuntimeWarning: overflow encountered in exp
  exp = np.exp(z)


(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([inf], dtype=float32)>,
 [<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>])

In [377]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [378]:
#Custom Training Loops
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [379]:

def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [380]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics,
          end=end)

In [381]:
import time

mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 - loss: 0.0900 - mean_square: 858.5000


In [382]:
def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)


progress_bar(3500, 10000, size=6)

' 3500/10000 [=>....]'

In [383]:
def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)

mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 [==============================] - loss: 0.0900 - mean_square: 858.5000


In [384]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [385]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.MeanSquaredError()
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)

Epoch 1/5
11610/11610 [==============================] - mean: 1.5378 - mean_absolute_error: 0.59566
Epoch 2/5
11610/11610 [==============================] - mean: 1.1032 - mean_absolute_error: 0.5625
Epoch 3/5
11610/11610 [==============================] - mean: 0.9469 - mean_absolute_error: 0.5484
Epoch 4/5
11610/11610 [==============================] - mean: 0.8717 - mean_absolute_error: 0.5421
Epoch 5/5
11610/11610 [==============================] - mean: 0.8282 - mean_absolute_error: 0.5393
